# Analysis of Seed Matchups

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [53]:
df = pd.read_csv("data/MNCAATourneyDetailedResults.csv")
df_teams = pd.read_csv("data/MTeams.csv")
df_conf = pd.read_csv("data/MTeamConferences.csv")
df_seeds = pd.read_csv("data/MNCAATourneySeeds.csv")

In [54]:
# get winning team's name: merge winning team id with team names
df = pd.merge(df, df_teams, how = "left", left_on = "WTeamID", right_on = "TeamID").\
                rename(columns = {"TeamName": "WTeamName"}).\
                drop(columns = ["TeamID", "FirstD1Season", "LastD1Season"])

# get losing team's name: merge losing team id with team names
df = pd.merge(df, df_teams, how = "left", left_on = "LTeamID", right_on = "TeamID").\
                rename(columns = {"TeamName": "LTeamName"}).\
                drop(columns = ["TeamID", "FirstD1Season", "LastD1Season"])

# get winning team's conference for season: merge with conference data using winning team id and season
df = pd.merge(df, df_conf, how = "left", left_on = ["WTeamID", "Season"], right_on = ["TeamID", "Season"]).\
                rename(columns = {"ConfAbbrev": "WConf"}).\
                drop(columns = ["TeamID"])

# get losing team's conference for season: merge with conference data using losing team id and season
df = pd.merge(df, df_conf, how = "left", left_on = ["LTeamID", "Season"], right_on = ["TeamID", "Season"]).\
                rename(columns = {"ConfAbbrev": "LConf"}).\
                drop(columns = ["TeamID"])

# get winning team's tourney seed for season: merge with seed data using winning team id and season
df = pd.merge(df, df_seeds, how = "left", left_on = ["WTeamID", "Season"], right_on = ["TeamID", "Season"]).\
                rename(columns = {"Seed": "WSeed"}).\
                drop(columns = ["TeamID"])

# get losing team's tourney seed for season: merge with seed data using losing team id and season
df = pd.merge(df, df_seeds, how = "left", left_on = ["LTeamID", "Season"], right_on = ["TeamID", "Season"]).\
                rename(columns = {"Seed": "LSeed"}).\
                drop(columns = ["TeamID"])

# separate the region id from the Seed value
df['WRegion'] = df['WSeed'].apply(lambda x: x[0])
df['LRegion'] = df['LSeed'].apply(lambda x: x[0])

df['WSeed'] = df['WSeed'].apply(lambda x: int(x[1:3]))
df['LSeed'] = df['LSeed'].apply(lambda x: int(x[1:3]))

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115 entries, 0 to 1114
Data columns (total 42 columns):
Season       1115 non-null int64
DayNum       1115 non-null int64
WTeamID      1115 non-null int64
WScore       1115 non-null int64
LTeamID      1115 non-null int64
LScore       1115 non-null int64
WLoc         1115 non-null object
NumOT        1115 non-null int64
WFGM         1115 non-null int64
WFGA         1115 non-null int64
WFGM3        1115 non-null int64
WFGA3        1115 non-null int64
WFTM         1115 non-null int64
WFTA         1115 non-null int64
WOR          1115 non-null int64
WDR          1115 non-null int64
WAst         1115 non-null int64
WTO          1115 non-null int64
WStl         1115 non-null int64
WBlk         1115 non-null int64
WPF          1115 non-null int64
LFGM         1115 non-null int64
LFGA         1115 non-null int64
LFGM3        1115 non-null int64
LFGA3        1115 non-null int64
LFTM         1115 non-null int64
LFTA         1115 non-null int64
L

In [56]:
mycols = ['Season','DayNum','NumOT','WSeed','LSeed','WScore','LScore','WConf','LConf','WTeamName','LTeamName',
          'WFGM','LFGM','WFGM3','LFGM3','WFTM','LFTM','WOR','LOR','WDR','LDR','WAst','LAst','WTO','LTO','WStl','LStl',
         'WBlk','LBlk','WPF','LPF','WRegion','LRegion','WTeamID','LTeamID']
df = df[mycols]

In [57]:
df.head()

,Season,DayNum,NumOT,WSeed,LSeed,WScore,LScore,WConf,LConf,WTeamName,...,WStl,LStl,WBlk,LBlk,WPF,LPF,WRegion,LRegion,WTeamID,LTeamID
0,2003,134,1,16,16,92,84,big_south,swac,UNC Asheville,...,5,5,3,0,22,22,X,X,1421,1411
1,2003,136,0,1,16,80,51,pac_ten,aec,Arizona,...,10,10,7,3,8,15,Z,Z,1112,1436
2,2003,136,0,10,7,84,71,pac_ten,cusa,Arizona St,...,7,2,4,5,19,18,Z,Z,1113,1272
3,2003,136,0,11,6,79,73,mac,mvc,C Michigan,...,13,6,1,6,19,21,Z,Z,1141,1166
4,2003,136,1,8,9,76,74,pac_ten,acc,California,...,8,5,2,8,14,19,W,W,1143,1301


In [58]:
df['DSeed'] = df['WSeed'] - df['LSeed']
df['DScore'] = df['WScore'] - df['LScore']
df['DFGM'] = df['WFGM'] - df['LFGM']
df['DFGM3'] = df['WFGM3'] - df['LFGM3']
df['DFTM'] = df['WFTM'] - df['LFTM']
df['DOR'] = df['WOR'] - df['LOR']
df['DDR'] = df['WDR'] - df['LDR']
df['DAst'] = df['WAst'] - df['LAst']
df['DTO'] = df['WTO'] - df['LTO']
df['DStl'] = df['WStl'] - df['LStl']
df['DBlk'] = df['WBlk'] - df['LBlk']
df['DPF'] = df['WPF'] - df['LPF']

In [59]:
df.head().T

,0,1,2,3,4
Season,2003,2003,2003,2003,2003
DayNum,134,136,136,136,136
NumOT,1,0,0,0,1
WSeed,16,1,10,11,8
LSeed,16,16,7,6,9
WScore,92,80,84,79,76
LScore,84,51,71,73,74
WConf,big_south,pac_ten,pac_ten,mac,pac_ten
LConf,swac,aec,cusa,mvc,acc
WTeamName,UNC Asheville,Arizona,Arizona St,C Michigan,California


Add round number using the DayNum and the SeedRoundSlots csv.

In [61]:
df_seeds = pd.read_csv("data/MNCAATourneySeedRoundSlots.csv")
df_seeds

In [63]:
df_rounds = pd.read_csv("data/MNCAATourneySeedRoundSlots.csv")
day1 = df_rounds.groupby(['GameRound', 'EarlyDayNum']).count().reset_index().drop(columns=['Seed', 'GameSlot', 'LateDayNum'])
day2 = df_rounds.groupby(['GameRound', 'LateDayNum']).count().reset_index().drop(columns=['Seed', 'GameSlot', 'EarlyDayNum'])
day1.columns = ['GameRound','DayNum']
day2.columns = ['GameRound','DayNum']

df_rounds = pd.concat([day1, day2], axis=0)
df = df.merge(df_rounds, how='left', on='DayNum')

Add "upset" boolean field. 

In [75]:
df['Upset'] = np.where(df.DSeed > 0, 1, 0)

## Explore Upsets

In [80]:
pd.crosstab(df.WConf, df.Upset, margins=True, normalize='index')

Upset,0,1
WConf,,
a_sun,0.200000,0.800000
a_ten,0.585366,0.414634
aac,0.631579,0.368421
acc,0.873626,0.126374
aec,0.500000,0.500000
big_east,0.769231,0.230769
big_sky,0.000000,1.000000
big_south,0.750000,0.250000
big_ten,0.756410,0.243590
